# zad2 Jakub Iliński

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List
import torch
import numpy as np
from torch.nn import functional as F
from typing import List

PAPUGA = 'flax-community/papuGaPT2'
DEVICE = "cuda"

tokenizer = AutoTokenizer.from_pretrained(PAPUGA)
model = AutoModelForCausalLM.from_pretrained(PAPUGA).to(DEVICE)
model.device

/home/hinski2/miniconda3/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

In [2]:

def compute_sentence_prob(sentence: List[str]) -> float:
        def _log_probs_from_logits(logits, labels):
            logp = F.log_softmax(logits, dim=-1)
            logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
            return logp_label
        
        def _sentence_prob(text: str):
            input_ids = tokenizer(text, return_tensors='pt')['input_ids'].to(DEVICE)
            with torch.no_grad():
                output = model(input_ids=input_ids)
                log_probs = _log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
                
            return float(log_probs.mean().item())
        
        return _sentence_prob(" ".join(sentence))

compute_sentence_prob(["wprost", "uwielbiała", "słuchać", "o", "wartościach", "własnych", "macierzy"])

-6.97428035736084